In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Daten laden
df = pd.read_csv('data.csv')

print("Ursprüngliche Datensatz-Info:")
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print(f"Missing values:\n{df.isnull().sum()}")


Ursprüngliche Datensatz-Info:
Shape: (9334, 23)
Columns: ['id', 'Datum', 'Umsatz', 'KielerWoche', 'Bewoelkung', 'Temperatur', 'Windgeschwindigkeit', 'Wettercode', 'ist_feiertag', 'Feiertag', 'feiertag_vortag', 'feiertag_folgetag', 'Wochentag', 'Monat', 'Jahr', 'Jahreszeit', 'Wettercode_fehlt', 'Warengruppe_Brot', 'Warengruppe_Brötchen', 'Warengruppe_Croissant', 'Warengruppe_Konditorei', 'Warengruppe_Kuchen', 'Warengruppe_Saisonbrot']
Missing values:
id                           0
Datum                        0
Umsatz                       0
KielerWoche                  0
Bewoelkung                  70
Temperatur                  16
Windgeschwindigkeit         16
Wettercode                2325
ist_feiertag                 0
Feiertag                  9133
feiertag_vortag              0
feiertag_folgetag            0
Wochentag                    0
Monat                        0
Jahr                         0
Jahreszeit                   0
Wettercode_fehlt             0
Warengruppe_Brot   

In [2]:
# 1. ID-Format umwandeln von YYYYMMDDW zu YYMMDDW
def convert_id_format(id_val):
    """Konvertiert ID von 1307011 (YYYYMMDDW) zu 1307011 (YYMMDDW)"""
    id_str = str(id_val)
    # Extrahiere Jahr (erste 4 Stellen), Monat-Tag (nächste 4), Warengruppe (letzte Stelle(n))
    year = id_str[:4]
    month_day = id_str[4:8]
    warengruppe = id_str[8:]
    
    # Konvertiere Jahr zu 2-stellig (z.B. 2013 -> 13)
    year_short = year[2:]
    
    return int(year_short + month_day + warengruppe)

df['id'] = df['id'].apply(convert_id_format)
print(f"\nID-Format geändert. Beispiel: {df['id'].head().tolist()}")


ID-Format geändert. Beispiel: [7011, 7013, 7015, 7012, 7014]


In [3]:
# 2. Datum in datetime konvertieren
df['Datum'] = pd.to_datetime(df['Datum'])

In [4]:
# 3. Fehlende Werte behandeln
print(f"\nFehlende Werte vor Bereinigung:\n{df.isnull().sum()}")

# Numerische Spalten: Median-Imputation (außer Wettercode)
numeric_cols = ['Bewoelkung', 'Temperatur', 'Windgeschwindigkeit']
for col in numeric_cols:
    if df[col].isnull().sum() > 0:
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)
        print(f"  {col}: {df[col].isnull().sum()} fehlende Werte mit Median ({median_val:.2f}) aufgefüllt")

# Wettercode: Eigene Kategorie '99' für fehlende Werte
if df['Wettercode'].isnull().sum() > 0:
    missing_count = df['Wettercode'].isnull().sum()
    df['Wettercode'] = df['Wettercode'].fillna(99)
    print(f"  Wettercode: {missing_count} fehlende Werte mit Kategorie '99' aufgefüllt")

# Kategorische Spalten: 'Kein_Feiertag' für fehlende Feiertage
categorical_cols = ['Feiertag']
for col in categorical_cols:
    if df[col].isnull().sum() > 0:
        missing_count = df[col].isnull().sum()
        df[col] = df[col].fillna('Kein_Feiertag')
        print(f"  {col}: {missing_count} fehlende Werte mit 'Kein_Feiertag' aufgefüllt")

print(f"\nFehlende Werte nach Bereinigung:\n{df.isnull().sum()}")
print(f"Alle fehlenden Werte behandelt: {not df.isnull().any().any()}")

# 4. One-Hot Encoding für kategorische Variablen
categorical_features = ['Wochentag', 'Monat', 'Jahreszeit', 'Feiertag']

# Erstelle Dummy-Variablen
df_encoded = pd.get_dummies(df, columns=categorical_features, prefix=categorical_features, drop_first=True)

# 5. Zielvariable definieren
target = 'Umsatz'

# 6. Features auswählen (alle außer ID, Datum, Umsatz)
feature_cols = [col for col in df_encoded.columns if col not in ['id', 'Datum', target]]

print(f"\nFeatures nach Encoding ({len(feature_cols)}):")
for i, col in enumerate(feature_cols):
    print(f"{i+1:2d}. {col}")

# 7. Datentypen überprüfen und korrigieren
print(f"\nDatentypen:")
print(df_encoded[feature_cols + [target]].dtypes.value_counts())

# Boolean zu int konvertieren
bool_cols = df_encoded.select_dtypes(include=['bool']).columns
df_encoded[bool_cols] = df_encoded[bool_cols].astype(int)

# 8. Features und Target separieren
X = df_encoded[feature_cols]
y = df_encoded[target]

print(f"\nFinale Datensets:")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"Keine fehlenden Werte in X: {not X.isnull().any().any()}")
print(f"Keine fehlenden Werte in y: {not y.isnull().any()}")

# 9. Übersicht der bereinigten Daten
print(f"\nStatistische Übersicht der Zielvariable:")
print(y.describe())

print(f"\nErste 5 Zeilen der Features:")
print(X.head())

# Optional: Skalierung vorbereiten (noch nicht angewendet)
scaler = StandardScaler()
print(f"\nScaler initialisiert (bereit für Anwendung auf Train-Set)")

print(f"\n{'='*50}")
print("DATENBEREINIGUNG ABGESCHLOSSEN")
print(f"{'='*50}")


Fehlende Werte vor Bereinigung:
id                           0
Datum                        0
Umsatz                       0
KielerWoche                  0
Bewoelkung                  70
Temperatur                  16
Windgeschwindigkeit         16
Wettercode                2325
ist_feiertag                 0
Feiertag                  9133
feiertag_vortag              0
feiertag_folgetag            0
Wochentag                    0
Monat                        0
Jahr                         0
Jahreszeit                   0
Wettercode_fehlt             0
Warengruppe_Brot             0
Warengruppe_Brötchen         0
Warengruppe_Croissant        0
Warengruppe_Konditorei       0
Warengruppe_Kuchen           0
Warengruppe_Saisonbrot       0
dtype: int64
  Bewoelkung: 0 fehlende Werte mit Median (6.00) aufgefüllt
  Temperatur: 0 fehlende Werte mit Median (11.62) aufgefüllt
  Windgeschwindigkeit: 0 fehlende Werte mit Median (10.00) aufgefüllt
  Wettercode: 2325 fehlende Werte mit Kategorie '9

In [5]:
# 10. Bereinigte Daten abspeichern
df_encoded.to_csv('data_clean.csv', index=False)
print(f"\n✓ Bereinigte Daten erfolgreich gespeichert als 'data_clean.csv'")


✓ Bereinigte Daten erfolgreich gespeichert als 'data_clean.csv'
